# notebook to generate inputs for EMSE Anylogic simulation

In [1]:
# objective: we want to have a table with one row per order, and first 4 columns contain the station number for each stop (up to 4 stops, and 0 means there is )
# no stop). Then a column with interarrival time (in seconds), then 48 columns (one per station), with the processing time for that station. 

In [19]:
#1- let's generate a table with 1000 orders
import pandas as pd
import numpy as np

def assign_next_stop(x,previous_stop_name,nb_stations): 
    next_stop=0
    if 0<x[previous_stop_name] < nb_stations: 
        next_stop = np.minimum(np.random.randint(x[previous_stop_name]+1,nb_stations+1,1)[0],np.random.randint(x[previous_stop_name]+1,nb_stations+1,1)[0])
    return next_stop


In [20]:
nb_stations = 6
nb_boxes = 1000
#create 1000 boxes
df = pd.DataFrame({'box':np.arange(1,nb_boxes+1)})
#assign stations to each stop
df['stop1']=np.minimum(np.random.randint(1,nb_stations,nb_boxes),np.random.randint(1,nb_stations,nb_boxes))
df['stop2']=df.apply(lambda x: assign_next_stop(x,'stop1',nb_stations=nb_stations), axis=1)
df['stop3']=df.apply(lambda x: assign_next_stop(x,'stop2',nb_stations=nb_stations), axis=1)
df['stop4']=df.apply(lambda x: assign_next_stop(x,'stop3',nb_stations=nb_stations), axis=1)
#define interarrival time
df['interarrival time'] = np.random.randint(1,10,nb_boxes)

In [21]:
#now creating N columns with random process times between 1 and 10 seconds
#we need to create a table with 1000 boxes, and for each box as many rows as stations
df_process_times = pd.DataFrame({'box':np.arange(1,nb_boxes+1)})
df_process_times['dummy_merge']=1
df_stations = pd.DataFrame({'station_num':np.arange(1,nb_stations+1)})
#df_stations['station_num'] = df_stations['station_num']
df_stations['dummy_merge']=1
#we use a dummy merge to merge all boxes to all stations
df_station_process_time = df_process_times.merge(df_stations,on='dummy_merge')
#and generate our process times
df_station_process_time['process_time']=np.random.randint(1,10,nb_stations*nb_boxes)
#then before putting all rows as columns, we create the column names
df_station_process_time['station_name'] = 'Station'+df_station_process_time['station_num'].astype(str).str.zfill(2)+'_time'
#and finally we pivot all rows into columns
df_station_process_time['station_num'] = df_station_process_time['station_num'].astype(int)

In [22]:
#a bit of reshaping magic
df3 = df[['box','stop1','stop2','stop3','stop4']].melt(id_vars='box').rename(columns={'variable':'stop_number','value':'station_num'}).sort_values('box')
df4 = df3.merge(df_station_process_time,on=['box','station_num']).rename(columns={'stop_number':'stop_name'})
df4 = df4.merge(pd.DataFrame({'stop_name':['stop1','stop2','stop3','stop4'],'stop_number':[1,2,3,4]})).sort_values(['box','stop_number'])

In [23]:
#final output shaping
df_simulation_input_format1 = df.merge(df4.pivot(index='box', columns='station_name', values='process_time').reset_index().fillna(0),on='box')
df_simulation_input_format2 = df4[['box','stop_number','station_num','process_time']]

In [24]:
df_simulation_input_format1.to_csv('df_simulation_input_format1.csv')
df_simulation_input_format2.to_csv('df_simulation_input_format2.csv')